In [1]:
import MyDL
import MyDL.nn as nn


In [4]:

a = MyDL.MyTensor([[1,2,3], [4,5,6]])
b = MyDL.MyTensor([1,2,3])
c = b * a
d = c.sum()
d.backward()
b.grad

array([5, 7, 9])

In [ ]:
import MyDL
import MyDL.nn as nn
import cupy as np
X = MyDL.MyTensor(np.random.rand(8, 3, 7, 7))
layer_conv = nn.Conv2D(3, 5, 4, 1, 2, False)
t = layer_conv(X)
print(t.shape)
t.backward()

(8, 5, 8, 8)


In [9]:
import numpy as np

a = [1,2,3]

a = np.asarray(a)
a

array([1, 2, 3])

In [ ]:
import cupy as np

print(np.__name__)

with np.load('figure\model_params\MLP3_(100,10)_relu_L2-0.0_lr-0.1.npz') as weights:
    print(len(weights.npz_file))
    for data in weights.npz_file.values():
        print(data.shape)

dd = {'ds': 1, 'ds2': 2}
aa = {k: v for k, v in dd.items() if k == 'ds'}
        

cupy
12
(784, 100)
(100,)
(100, 10)
(10,)
(10, 10)
(10,)
(784,)
(784,)
(100,)
(100,)
(10,)
(10,)


In [ ]:
# x = cp.arange(6).reshape(2, 3).astype('f')
# print(x)
import os
os.add_dll_directory(r"C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8\bin")
import cupy as cp
from cupy.lib.stride_tricks import as_strided
print("cupy version:", cp.__version__)
print("CUDA runtime version:", cp.cuda.runtime.runtimeGetVersion())
print("Device properties:", cp.cuda.runtime.getDeviceProperties(0))
x = cp.arange(6).reshape(2, 3).astype('f')

a = cp.arange(16).reshape(4, 4)
view = as_strided(a, shape=(2, 2, 2, 2), strides=(16, 4, 16, 4))
print(view)



cupy version: 13.4.1
CUDA runtime version: 12080
Device properties: {'name': b'NVIDIA GeForce RTX 3060 Laptop GPU', 'totalGlobalMem': 6441926656, 'sharedMemPerBlock': 49152, 'regsPerBlock': 65536, 'warpSize': 32, 'maxThreadsPerBlock': 1024, 'maxThreadsDim': (1024, 1024, 64), 'maxGridSize': (2147483647, 65535, 65535), 'clockRate': 1425000, 'totalConstMem': 65536, 'major': 8, 'minor': 6, 'textureAlignment': 512, 'texturePitchAlignment': 32, 'multiProcessorCount': 30, 'kernelExecTimeoutEnabled': 1, 'integrated': 0, 'canMapHostMemory': 1, 'computeMode': 0, 'maxTexture1D': 131072, 'maxTexture2D': (131072, 65536), 'maxTexture3D': (16384, 16384, 16384), 'concurrentKernels': 1, 'ECCEnabled': 0, 'pciBusID': 1, 'pciDeviceID': 0, 'pciDomainID': 0, 'tccDriver': 0, 'memoryClockRate': 7001000, 'memoryBusWidth': 192, 'l2CacheSize': 3145728, 'maxThreadsPerMultiProcessor': 1536, 'isMultiGpuBoard': 0, 'cooperativeLaunch': 1, 'cooperativeMultiDeviceLaunch': 0, 'deviceOverlap': 1, 'maxTexture1DMipmap': 32

In [ ]:
import cupy as np
from cupyx.lib.stride_tricks import as_strided
import MyDL
import MyDL.nn

np.random.seed(42)

conv_layer = MyDL.nn.layers.Conv2D(in_channels=3, out_channels=5, kernel_size=4, stride=2, padding=(2, 1, 2, 1), bias=True)

X = np.random.rand(8, 3, 7, 7)   # 输入通道 C_in = 3
X = MyDL.MyTensor(X)
x_out = conv_layer(X)
x_out.shape
y = x_out.sum()
y.backward()


print(conv_layer.kernel.grad)


ModuleNotFoundError: No module named 'cupyx.lib'

In [14]:
def ll(*nums):
    print(type(nums))
    return nums

ll(1,2,3,4)

<class 'tuple'>


(1, 2, 3, 4)

In [ ]:
# 单通道卷积
import cupy as np
from cupyx.lib.stride_tricks import as_strided

def conv2d_strided(X, K, padding=0, stride=1):
    """
    用 as_strided 实现高效的二维卷积（单通道）
    参数：
        X : 原始图像 (H, W)
        K : 卷积核 (kH, kW)
        padding : 填充边界（默认0）
        stride : 步幅（默认1）
    返回：
        Y : 卷积结果
    """
    X = np.array(X)
    K = np.array(K)

    H, W = X.shape
    kH, kW = K.shape

    # 添加 zero-padding
    if padding > 0:
        X_padded = np.pad(X, ((padding, padding), (padding, padding)), mode='constant')
    else:
        X_padded = X

    H_p, W_p = X_padded.shape

    # 输出尺寸
    out_h = (H_p - kH) // stride + 1
    out_w = (W_p - kW) // stride + 1

    # 构造滑动窗口视图：(out_h, out_w, kH, kW)
    shape = (out_h, out_w, kH, kW)
    strides = (
        X_padded.strides[0] * stride,
        X_padded.strides[1] * stride,
        X_padded.strides[0],
        X_padded.strides[1],
    )
    X_strided = as_strided(X_padded, shape=shape, strides=strides)

    # 逐窗口点乘卷积核并求和（向量化）
    Y = np.einsum('ijkl,kl->ij', X_strided, K)

    return Y

X = np.array([
    [1, 2, 3, 0],
    [0, 1, 2, 1],
    [1, 0, 2, 1],
    [2, 1, 3, 0]
])

K = np.array([
    [1, 0, -1],
    [1, 0, -1],
    [1, 0, -1]
])

Y = conv2d_strided(X, K, padding=1, stride=1)
print(Y)


[[-3 -4  2  5]
 [-3 -5  1  7]
 [-2 -4  0  7]
 [-1 -2  0  5]]


In [ ]:
# 多通道卷积
import cupy as np
from cupyx.lib.stride_tricks import as_strided

def conv2d_multi_channel(X, K, padding=0, stride=1):
    """
    多通道二维卷积实现（无for循环）
    参数：
        X: 输入图像 (C_in, H, W)
        K: 卷积核 (C_out, C_in, kH, kW)
        padding: 零填充大小
        stride: 步幅
    返回：
        Y: 输出特征图 (C_out, H_out, W_out)
    """
    C_in, H, W = X.shape
    C_out, C_in_k, kH, kW = K.shape
    assert C_in == C_in_k, "卷积核的输入通道数必须等于输入图像的通道数"

    # padding 每个通道
    if padding > 0:
        X_padded = np.pad(X, ((0, 0), (padding, padding), (padding, padding)), mode='constant')
    else:
        X_padded = X

    H_p, W_p = X_padded.shape[1], X_padded.shape[2]

    H_out = (H_p - kH) // stride + 1
    W_out = (W_p - kW) // stride + 1

    # 滑动窗口展开：输出形状 (C_in, H_out, W_out, kH, kW)
    strides = X_padded.strides
    new_shape = (C_in, H_out, W_out, kH, kW)
    new_strides = (
        strides[0],
        strides[1] * stride,
        strides[2] * stride,
        strides[1],
        strides[2]
    )
    X_strided = as_strided(X_padded, shape=new_shape, strides=new_strides)

    # 卷积操作：K 是 (C_out, C_in, kH, kW)，X 是 (C_in, H_out, W_out, kH, kW)
    # 使用einsum进行高效批量计算
    Y = np.einsum('oimn,ihwmn->ohw', K, X_strided)

    return Y

X = np.random.rand(3, 5, 5)   # 输入通道 C_in = 3
K = np.random.rand(4, 3, 3, 3) # 输出通道 C_out = 4，卷积核 3x3

Y = conv2d_multi_channel(X, K, padding=1, stride=1)

print(Y.shape)  # 应输出 (4, 5, 5)


(4, 5, 5)


In [ ]:
# 多通道卷积
import cupy as np
from typing import Union, Tuple
from cupyx.lib.stride_tricks import as_strided

def conv2d_multi_channel(X, K, padding:Union[int, Tuple[int, int, int, int]]=0, 
                         stride:Union[int, Tuple[int, int]]=1):
    """
    多通道二维卷积实现（无for循环）
    参数：
        X: 输入图像 (batch, C_in, H, W)
        K: 卷积核 (C_out, C_in, kH, kW)
        padding: 零填充大小，整数或元组(up, down, left, right)
        stride: 步幅
    返回：
        Y: 输出特征图 (C_out, H_out, W_out)
    """

    batch, C_in, H, W = X.shape
    C_out, C_in_k, kH, kW = K.shape
    assert C_in == C_in_k, "卷积核的输入通道数必须等于输入图像的通道数"

    # padding 每个通道
    if isinstance(padding, int):
        X_padded = np.pad(X, ((0, 0), (0, 0), (padding, padding), (padding, padding)), mode='constant')
    else:
        X_padded = np.pad(X, ((0, 0), (0, 0), (padding[0], padding[1]), (padding[2], padding[3])), mode='constant')
    
    if isinstance(stride, int):
        stride = (stride, stride)

    H_p, W_p = X_padded.shape[2], X_padded.shape[3]

    H_out = (H_p - kH) // stride[0] + 1
    W_out = (W_p - kW) // stride[1] + 1

    # 滑动窗口展开：输出形状 (C_in, H_out, W_out, kH, kW)
    strides = X_padded.strides
    new_shape = (batch, C_in, H_out, W_out, kH, kW)
    new_strides = (
        strides[0],
        strides[1],
        strides[2] * stride[0],
        strides[3] * stride[1],
        strides[2],
        strides[3]
    )
    X_strided = as_strided(X_padded, shape=new_shape, strides=new_strides)

    # 卷积操作：K 是 (C_out, C_in, kH, kW)，X 是 (batch, C_in, H_out, W_out, kH, kW)
    # 使用einsum进行高效批量计算
    Y = np.einsum('oimn,bihwmn->bohw', K, X_strided)

    return Y


X = np.random.rand(8, 3, 7, 7)   # 输入通道 C_in = 3
K = np.random.rand(5, 3, 4, 4)   # c_in, c_out, kh, kw

Y = conv2d_multi_channel(X, K, padding=(2,1,2,1), stride=(2,2))

print(X.shape)
print(Y.shape)  # 应输出 (8, 4, 5, 5)

(8, 3, 7, 7)
(8, 5, 4, 4)


In [ ]:
import cupy as np
a = np.array([[1,2,3],[4,5,6]])
a[:, ::2] = 0
a

array([[0, 2, 0],
       [0, 5, 0]])